# **Tuwaiq: Data Data Science and Machine Learning**

**Week #3 - Day #15**

Project: Pandas and Geopandas (Logistics).

Part #1: Exploratory Data Analysis (EDA).

#####Dataset link (Kaggle): New york city - Airbnb
https://www.kaggle.com/datasets/dgomonov/new-york-city-airbnb-open-data?select=AB_NYC_2019.csv

## Geopandas

In [1]:
# Set up for geopandas and the maps
!pip install mapclassify -q

import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import HeatMap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 882.2/882.2 kB 12.2 MB/s eta 0:00:00


In [3]:
# Load the cleaned Airbnb NYC dataset
df = pd.read_csv("AB_NYC_2019_cleaned.csv")

print("Data sample:")
df.head()

Data sample:


id                                              name  host_id  \
0  2539                Clean & quiet apt home by the park     2787   
1  2595                             Skylit Midtown Castle     2845   
2  3647               THE VILLAGE OF HARLEM....NEW YORK !     4632   
3  3831                   Cozy Entire Floor of Brownstone     4869   
4  5022  Entire Apt: Spacious Studio/Loft by central park     7192   

     host_name neighbourhood_group neighbourhood  latitude  longitude  \
0         John            Brooklyn    Kensington  40.64749  -73.97237   
1     Jennifer           Manhattan       Midtown  40.75362  -73.98377   
2    Elisabeth           Manhattan        Harlem  40.80902  -73.94190   
3  LisaRoxanne            Brooklyn  Clinton Hill  40.68514  -73.95976   
4        Laura           Manhattan   East Harlem  40.79851  -73.94399   

         room_type  price  ...  last_review  reviews_per_month  \
0     Private room  149.0  ...   2018-10-19               0.21   
1  Entire home/apt  225.0  ...   2019-05-21               0.38   
2     Private room  150.0  ...          NaN               0.00   
3  Entire home/apt   89.0  ...   2019-07-05               4.64   
4  Entire home/apt   80.0  ...   2018-11-19               0.10   

  calculated_host_listings_count  availability_365  last_review_year  \
0                            6.0             365.0            2018.0   
1                            2.0             355.0            2019.0   
2                            1.0             365.0               NaN   
3                            1.0             194.0            2019.0   
4                            1.0               0.0            2018.0   

   last_review_month  last_review_day  price_log1p  avg_price_neigh_group  \
0               10.0             19.0     5.010635             124.383207   
1                5.0             21.0     5.420535             196.875814   
2                NaN              NaN     5.017280             196.875814   
3                7.0              5.0     4.499810             124.383207   
4               11.0             19.0     4.394449             196.875814   

   has_review  
0         1.0  
1         1.0  
2         0.0  
3         1.0  
4         1.0  

[5 rows x 22 columns]

In [3]:
# Column names in the dataset
print("Columns:", df.columns)

Columns: Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'last_review_year', 'last_review_month',
       'last_review_day', 'price_log1p', 'avg_price_neigh_group',
       'has_review'],
      dtype='object')


In [4]:
# Convert the DataFrame to a GeoDataFrame using longitude and latitude
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df['longitude'], df['latitude']),
    crs="EPSG:4326"  # Geographic coordinate system
)

In [5]:
# Exclude 'geometry' from tooltip columns
tooltip_cols = [col for col in gdf.columns if col != "geometry"]

# Explore with tooltip excluding geometry
map_explore = gdf.explore(
    column=None,
    cmap="Blues",
    tiles="CartoDB positron",
    tooltip=tooltip_cols,       # Now safe for Folium
    marker_kwds={"radius": 5}
)

# Display the map
map_explore

In [6]:
# Create a new Folium map
m = folium.Map(
    location=[gdf['latitude'].mean(), gdf['longitude'].mean()],  # Center the map
    zoom_start=10
)

# Prepare data for the heatmap as a list of [lat, lon] pairs
heat_data = gdf[['latitude', 'longitude']].values.tolist()

# Add heatmap layer to the map
HeatMap(
    heat_data,
    radius=12,     # Radius around each point
    blur=9,       # Blurring effect
    max_zoom=13    # Max zoom level for heatmap effect
).add_to(m)

# Display the heatmap
m

ValueError: Location values cannot contain NaNs.

In [5]:
# Saving the file (GeoJSON)
gdf.to_file("AB_NYC.geojson", driver="GeoJSON")

print("GeoJSON file saved successfully!")

GeoJSON file saved successfully!


## Dashboard

In [6]:
# Install packages
!pip install streamlit pyngrok geopandas folium streamlit-folium matplotlib -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.4/524.4 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 139.4 MB/s eta 0:00:00


In [16]:
# Enabling the secret key
from google.colab import userdata
userdata.get('Wajd_Key')

'2vshh2i4PYUjwNGIkXnp3ar8zhP_5S5UynGZRaug9kZggPuL9'

In [7]:
!ngrok config add-authtoken 2vshh2i4PYUjwNGIkXnp3ar8zhP_5S5UynGZRaug9kZggPuL9 # Secret key

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [21]:
# Creating the file dashboard.py
%%writefile dashboard.py
import geopandas as gpd
import streamlit as st
import folium
from streamlit_folium import st_folium
import matplotlib.pyplot as plt
from folium.plugins import HeatMap, MarkerCluster
import plotly.express as px
import plotly.graph_objects as go

#--------------------------------------------------------------------

# Load Data
@st.cache_data
def load_data():
    gdf = gpd.read_file("AB_NYC.geojson")
    return gdf

gdf = load_data()

#--------------------------------------------------------------------

# Sidebar Filters
st.sidebar.title("Filters")

selected_group = st.sidebar.multiselect(
    "Select Neighbourhood Group:",
    options=gdf["neighbourhood_group"].unique(),
    default=gdf["neighbourhood_group"].unique()
)

selected_neighbourhood = st.sidebar.multiselect(
    "Select Neighbourhood:",
    options=gdf["neighbourhood"].unique(),
    default=gdf["neighbourhood"].unique()
)

selected_room = st.sidebar.multiselect(
    "Select Room Type:",
    options=gdf["room_type"].unique(),
    default=gdf["room_type"].unique()
)

price_range = st.sidebar.slider(
    "Price Range:",
    int(gdf["price"].min()),
    int(gdf["price"].max()),
    (50, 500)
)

min_nights_range = st.sidebar.slider(
    "Minimum Nights:",
    int(gdf["minimum_nights"].min()),
    int(gdf["minimum_nights"].max()),
    (1, 30)
)

availability_range = st.sidebar.slider(
    "Availability (days/year):",
    int(gdf["availability_365"].min()),
    int(gdf["availability_365"].max()),
    (0, 365)
)

reviews_range = st.sidebar.slider(
    "Number of Reviews:",
    int(gdf["number_of_reviews"].min()),
    int(gdf["number_of_reviews"].max()),
    (0, 50)
)

reviews_per_month_range = st.sidebar.slider(
    "Reviews per Month:",
    float(gdf["reviews_per_month"].min()),
    float(gdf["reviews_per_month"].max()),
    (0.0, 5.0)
)

host_listings_range = st.sidebar.slider(
    "Host Listings Count:",
    int(gdf["calculated_host_listings_count"].min()),
    int(gdf["calculated_host_listings_count"].max()),
    (1, 10)
)

#--------------------------------------------------------------------

# Applying the filters we created previously
df_filtered = gdf[
    (gdf["neighbourhood_group"].isin(selected_group)) &
    (gdf["neighbourhood"].isin(selected_neighbourhood)) &
    (gdf["room_type"].isin(selected_room)) &
    (gdf["price"].between(price_range[0], price_range[1])) &
    (gdf["minimum_nights"].between(min_nights_range[0], min_nights_range[1])) &
    (gdf["availability_365"].between(availability_range[0], availability_range[1])) &
    (gdf["number_of_reviews"].between(reviews_range[0], reviews_range[1])) &
    (gdf["reviews_per_month"].between(reviews_per_month_range[0], reviews_per_month_range[1])) &
    (gdf["calculated_host_listings_count"].between(host_listings_range[0], host_listings_range[1]))
]

#--------------------------------------------------------------------

# Tabs
tab1, tab2 = st.tabs(["Overview", "Maps"])

#--------------------------------------------------------------------

# Tab 1: Overview
with tab1:
    st.title("Airbnb NYC Dashboard - Overview")

#--------------------------------------------------------------------

    # Sample Data
    st.write("### Sample of Data")
    st.write("Displays a few rows of the filtered dataset to give an overview of the listings.")
    st.dataframe(df_filtered.head())

#--------------------------------------------------------------------

    # Summary Statistics
    st.write("### Summary Statistics")
    st.write("Provides statistical summary (mean, min, max, std) for numerical columns.")
    st.write(df_filtered.describe())

#--------------------------------------------------------------------

    # Histograms
    st.write("### Price Distribution (Histogram)")
    st.write("Shows how listing prices are distributed. Helps identify common price ranges.")
    fig_price = px.histogram(df_filtered, x="price", nbins=50, range_x=[0, 500], color_discrete_sequence=["skyblue"])
    st.plotly_chart(fig_price, use_container_width=True)

    st.write("### Minimum Nights Distribution (Histogram)")
    st.write("Displays how many nights listings typically require. Detects unusually long stays.")
    fig_nights = px.histogram(df_filtered, x="minimum_nights", nbins=50, color_discrete_sequence=["orange"])
    st.plotly_chart(fig_nights, use_container_width=True)

    st.write("### Availability Distribution (Histogram)")
    st.write("Shows the distribution of number of available days per year for listings.")
    fig_avail = px.histogram(df_filtered, x="availability_365", nbins=50, color_discrete_sequence=["green"])
    st.plotly_chart(fig_avail, use_container_width=True)

    st.write("### Number of Reviews Distribution (Histogram)")
    st.write("Shows how many reviews listings have. Helps identify active vs inactive listings.")
    fig_reviews = px.histogram(df_filtered, x="number_of_reviews", nbins=50, color_discrete_sequence=["purple"])
    st.plotly_chart(fig_reviews, use_container_width=True)

    st.write("### Reviews per Month Distribution (Histogram)")
    st.write("Shows review frequency per month. Useful for understanding listing activity.")
    fig_rpm = px.histogram(df_filtered, x="reviews_per_month", nbins=50, color_discrete_sequence=["pink"])
    st.plotly_chart(fig_rpm, use_container_width=True)


#--------------------------------------------------------------------

    # Bar Charts
    st.write("### Average Price by Neighbourhood Group (Bar Chart)")
    st.write("Compares average listing price across different neighbourhood groups.")
    avg_price_group = df_filtered.groupby("neighbourhood_group")["price"].mean().reset_index()
    fig_bar_group = px.bar(avg_price_group, x="neighbourhood_group", y="price", color="price", color_continuous_scale="Blues")
    st.plotly_chart(fig_bar_group, use_container_width=True)

    st.write("### Top 15 Neighbourhoods by Average Price (Bar Chart)")
    st.write("Shows the most expensive neighbourhoods on average.")
    avg_price_neigh = df_filtered.groupby("neighbourhood")["price"].mean().sort_values(ascending=False).head(15).reset_index()
    fig_bar_neigh = px.bar(avg_price_neigh, x="neighbourhood", y="price", color="price", color_continuous_scale="Oranges")
    st.plotly_chart(fig_bar_neigh, use_container_width=True)


#--------------------------------------------------------------------

    # Pie Charts
    st.write("### Room Type Distribution (Pie Chart)")
    st.write("Shows proportion of each room type in the filtered listings.")
    room_counts = df_filtered["room_type"].value_counts().reset_index()
    room_counts.columns = ["room_type", "count"]
    fig_pie_room = px.pie(room_counts, names="room_type", values="count", color_discrete_sequence=px.colors.qualitative.Set2)
    st.plotly_chart(fig_pie_room, use_container_width=True)

    st.write("### Neighbourhood Group Distribution (Pie Chart)")
    st.write("Shows proportion of listings in each neighbourhood group.")
    group_counts = df_filtered["neighbourhood_group"].value_counts().reset_index()
    group_counts.columns = ["neighbourhood_group", "count"]
    fig_pie_group = px.pie(group_counts, names="neighbourhood_group", values="count", color_discrete_sequence=px.colors.qualitative.Pastel)
    st.plotly_chart(fig_pie_group, use_container_width=True)

#--------------------------------------------------------------------

# Tab 2: Maps
with tab2:
    st.title("Maps")

    # ---------------- Marker Cluster Map ----------------
    st.write("### Marker Cluster Map")
    m_cluster = folium.Map(location=[40.7128, -74.0060], zoom_start=10)
    marker_cluster = MarkerCluster().add_to(m_cluster)
    for _, row in df_filtered.iterrows():
        folium.Marker(location=[row['latitude'], row['longitude']], popup=row['name']).add_to(marker_cluster)
    st_folium(m_cluster, width=700, height=500)

    # ---------------- Heatmap ----------------
    st.write("### Heatmap of Listings")
    heatmap = folium.Map(location=[40.7128, -74.0060], zoom_start=10)
    heat_data = [[row['latitude'], row['longitude']] for _, row in df_filtered.iterrows()]
    HeatMap(heat_data, radius=8).add_to(heatmap)
    st_folium(heatmap, width=700, height=500)

    # ---------------- Bubble Map: Price & Room Type ----------------
    st.write("### Bubble Map: Price vs Location by Room Type")
    bubble_map = folium.Map(location=[40.7128, -74.0060], zoom_start=10)
    room_colors = {"Entire home/apt":"blue", "Private room":"green", "Shared room":"red"}
    for _, row in df_filtered.iterrows():
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=max(row['price']/100, 3),
            color=room_colors.get(row['room_type'], "gray"),
            fill=True,
            fill_opacity=0.6,
            popup=f"{row['name']} | ${row['price']}"
        ).add_to(bubble_map)
    st_folium(bubble_map, width=700, height=500)

Overwriting dashboard.py


In [ ]:
'''



***Should be in Tab 2 But it's heavy load...



    # ---------------- Choropleth: Avg Price ----------------
    st.write("### Choropleth Map (Avg Price by Neighbourhood Group)")
    try:
        geojson_file = "AB_NYC.geojson"
        df_grouped = df_filtered.groupby("neighbourhood_group")["price"].mean().reset_index()
        choropleth_map = folium.Map(location=[40.7128, -74.0060], zoom_start=10)
        folium.Choropleth(
            geo_data=geojson_file,
            name="choropleth",
            data=df_grouped,
            columns=["neighbourhood_group", "price"],
            key_on="feature.properties.neighbourhood_group",
            fill_color="YlOrRd",
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name="Average Price"
        ).add_to(choropleth_map)
        st_folium(choropleth_map, width=700, height=500)
    except Exception as e:
        st.error(f"Error loading Choropleth Map: {e}")

    # ---------------- Choropleth: Count of Listings ----------------
    st.write("### Choropleth Map (Count of Listings per Neighbourhood)")
    try:
        df_count = df_filtered.groupby("neighbourhood")["id"].count().reset_index()
        df_count.columns = ["neighbourhood", "count"]
        choropleth_count = folium.Map(location=[40.7128, -74.0060], zoom_start=10)
        folium.Choropleth(
            geo_data=geojson_file,
            data=df_count,
            columns=["neighbourhood", "count"],
            key_on="feature.properties.neighbourhood",
            fill_color="PuBu",
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name="Number of Listings"
        ).add_to(choropleth_count)
        st_folium(choropleth_count, width=700, height=500)
    except Exception as e:
        st.error(f"Error loading Choropleth Count Map: {e}")

    # ---------------- Choropleth: Avg Reviews per Month ----------------
    st.write("### Choropleth Map (Avg Reviews per Month)")
    try:
        df_reviews = df_filtered.groupby("neighbourhood_group")["reviews_per_month"].mean().reset_index()
        choropleth_reviews = folium.Map(location=[40.7128, -74.0060], zoom_start=10)
        folium.Choropleth(
            geo_data=geojson_file,
            data=df_reviews,
            columns=["neighbourhood_group", "reviews_per_month"],
            key_on="feature.properties.neighbourhood_group",
            fill_color="YlGn",
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name="Average Reviews per Month"
        ).add_to(choropleth_reviews)
        st_folium(choropleth_reviews, width=700, height=500)
    except Exception as e:
        st.error(f"Error loading Reviews Choropleth Map: {e}")

'''

In [ ]:
from pyngrok import ngrok
import time

# close previous tunnels
ngrok.kill()

# Open an ngrok tunnel on port (8501) or (8502) - if (8501) Doesn't work
# This will give a public URL to access the Streamlit app
public_url = ngrok.connect(8501)
print("Public URL:", public_url)

# Run the Streamlit app in the background
# The '&' allows the app to run asynchronously
# 'sleep 10' gives Streamlit some time to start
!streamlit run dashboard.py & sleep 5

Public URL: NgrokTunnel: "https://739378cbc881.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.66.118.242:8501

2025-09-27 10:01:59.708 Serialization of dataframe to Arrow table was unsuccessful. Applying automatic fixes for column types to make the dataframe Arrow-compatible.
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/streamlit/dataframe_util.py", line 822, in convert_pandas_df_to_arrow_bytes
    table = pa.Table.from_pandas(df)
            ^^^^^^^^^^^^^^^^^^^^^^^^
  File "pyarrow/table.pxi", line 4751, in pyarrow.lib.Table.from_pandas
  File "/usr/local/lib/python3.12/dist-packages/pyarrow/pandas_compat.py", line 625, in dataframe_to_arrays
    arrays = [convert_column(c, f)
              ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pyarrow/pandas_compat.py", line 612, 